## 난임 환자 대상 임신 성공 여부 예측

### LGAimers 6th 온라인 해커톤

Import

In [1100]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [1101]:
import platform
from matplotlib import rc

# 운영 체제에 따라 폰트 설정
if platform.system() == 'Windows':  # Windows 환경
    rc('font', family='Malgun Gothic')  # 맑은 고딕

# 음수 표시 가능하도록 설정
plt.rcParams['axes.unicode_minus'] = False

In [1102]:
# 경고문구 숨기기
import warnings
import pandas as pd

warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)

반복적으로 쓰는 함수툴

In [1103]:
def plot_box(df, col_name):
    """
    주어진 데이터프레임과 열 이름에 대해 박스 플롯을 그리는 함수.

    Parameters:
    df (pd.DataFrame): 데이터프레임
    col_name (str): 열 이름
    """
    plt.figure(figsize=(7, 4))
    plt.boxplot(df[col_name], vert=False)
    plt.xlabel(col_name)
    plt.title(f'Box Plot of {col_name}')
    plt.show()

def value_counts_ratio(df, col_name, target_name='임신_성공_여부'):
    """
    주어진 데이터프레임의 특정 열에 대해 각 값마다 타겟 변수의 비율과 갯수, 총 갯수를 출력하는 함수.

    Parameters:
    df (pd.DataFrame): 데이터프레임
    col_name (str): 열 이름
    target_name (str): 타겟 변수 이름
    """
    # 각 값마다 타겟 변수의 비율 계산
    value_counts = df.groupby(col_name)[target_name].value_counts(normalize=True).unstack().fillna(0)
    
    # 각 값마다 타겟 변수의 갯수 계산
    counts = df.groupby(col_name)[target_name].value_counts().unstack().fillna(0)
    
    # 각 값마다 총 갯수 계산
    total_counts = df[col_name].value_counts().rename('Total_Count')
    
    # 비율과 갯수를 합침
    result = value_counts.join(counts, lsuffix='_ratio', rsuffix='_count')
    
    # 총 갯수를 합침
    result = result.join(total_counts, on=col_name)
    
    # 출력 형식 조정
    result.index.name = 'variable'
    print(f"\n{col_name}별 {target_name} 비율 및 갯수\n")
    print(result.rename(columns=lambda x: x.split('_')[0]))

def summarize_group(df, group_by_columns):
    """
    주어진 데이터프레임을 특정 열들로 그룹화하여 각 그룹의 '임신_성공_여부' 비율과 갯수를 요약하는 함수.

    Parameters:
    df (pd.DataFrame): 데이터프레임
    group_by_columns (list): 그룹화할 열 이름들의 리스트
    """
    # 데이터프레임을 그룹화
    grouped_df = df.groupby(group_by_columns)
    
    # 결과를 저장할 리스트 초기화
    results = []
    
    # 그룹화된 데이터프레임의 내용을 확인하는 코드
    for name, group in grouped_df:
        # 그룹의 갯수 계산
        group_count = group.shape[0]
        
        # '임신_성공_여부' 변수의 '임신 성공' 비율과 갯수 계산
        success_count = group['임신_성공_여부'].value_counts().get(1, 0)
        success_ratio = success_count / group_count
        
        # 결과 리스트에 추가
        results.append([name, success_count, success_ratio, group_count])
    
    # 결과 리스트를 데이터프레임으로 변환
    results_df = pd.DataFrame(results, columns=['group', "'임신_성공_여부' count", 'ratio', 'Total'])
    
    # 그룹화된 변수들의 이름을 제목행으로 출력
    print(f"Grouped by: {', '.join(group_by_columns)}")
    print()
    # 데이터프레임 출력
    print(results_df)

def plot_ratio(df, group_by_column, target_column='임신_성공_여부', success_value=1):
    """
    주어진 데이터프레임을 특정 열로 그룹화하여 각 그룹의 타겟 변수의 성공 비율을 시각화하는 함수.

    Parameters:
    df (pd.DataFrame): 데이터프레임
    group_by_column (str): 그룹화할 열 이름
    target_column (str): 타겟 변수 이름
    success_value (int): 타겟 변수의 성공을 나타내는 값
    """
    # 데이터프레임을 그룹화
    grouped_df = df.groupby(group_by_column)
    
    # 결과를 저장할 리스트 초기화
    results = []
    
    # 그룹화된 데이터프레임의 내용을 확인하는 코드
    for name, group in grouped_df:
        # 그룹의 갯수 계산
        group_count = group.shape[0]
        
        # '임신_성공_여부' 변수의 '성공' 비율과 갯수 계산
        success_count = group[target_column].value_counts().get(success_value, 0)
        success_ratio = success_count / group_count
        
        # 결과 리스트에 추가
        results.append([name, success_count, success_ratio, group_count])
    
    # 결과 리스트를 데이터프레임으로 변환
    results_df = pd.DataFrame(results, columns=['group', f"'{success_value}' count", 'ratio', 'Total'])
    
    # 그래프 크기 설정
    plt.figure(figsize=(10, 5))
    
    # 막대 그래프 생성
    ax = results_df.plot(kind='bar', x='group', y='ratio', legend=False)
    
    # 각 막대 위에 임신 성공 여부 갯수와 총 갯수 표시
    for i, (success_count, total) in enumerate(zip(results_df[f"'{success_value}' count"], results_df['Total'])):
        ax.text(i, results_df['ratio'][i], f'{success_count} ({total})', ha='center', va='bottom', fontsize=8)
    
    # 그래프 제목 및 축 레이블 설정
    ax.set_title(f'{success_value} Ratio by {group_by_column}')
    ax.set_xlabel(group_by_column)
    ax.set_ylabel(f'{success_value} Ratio')
   
    # 그래프 출력
    plt.show()

def plot_ratio_boxplot(data, column_name, target_column='임신_성공_여부'):
    """
    주어진 데이터프레임의 특정 열과 타겟 변수 간의 관계를 박스 플롯으로 시각화하는 함수.

    Parameters:
    data (pd.DataFrame): 데이터프레임
    column_name (str): 열 이름
    target_column (str): 타겟 변수 이름
    """

    # 그래프 그리기
    plt.figure(figsize=(10, 6))
    sns.boxplot(x=column_name, y=target_column, data=data)

    # 그래프 제목 및 레이블 설정
    plt.title(f'{column_name} vs {target_column}')
    plt.xlabel(column_name)
    plt.ylabel(target_column)

    # 그래프 출력
    plt.show()

### Data Load

In [1104]:
# 데이터 로드
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

# 변수명에서 공백을 밑줄로 대체
train.columns = train.columns.str.replace(' ', '_')
test.columns = test.columns.str.replace(' ', '_')

In [1105]:
# train.info(verbose=True)

## EDA

In [1106]:
# '시술_유형' 변수의 값이 'IVF'에 해당 안하는 데이터 추출
IVF_train = train[train['시술_유형'] == 'IVF'] 
IVF_test = test[test['시술_유형'] == 'IVF']

# 모든 값이 결측값인 변수를 제거
IVF_train = IVF_train.dropna(axis=1, how='all')
IVF_test = IVF_test.dropna(axis=1, how='all')

# 결측값을 제외한 고유값이 1개인 변수 제거
unique_value_columns_train = IVF_train.columns[IVF_train.nunique(dropna=True) <= 1]
unique_value_columns_test = IVF_test.columns[IVF_test.nunique(dropna=True) <= 1]

# 결측값이 없는 상태에서 고유값이 1개인 경우만 제거
IVF_train = IVF_train.drop(columns=[col for col in unique_value_columns_train if IVF_train[col].notna().all()])
IVF_test = IVF_test.drop(columns=[col for col in unique_value_columns_test if IVF_test[col].notna().all()])

# 결과를 원래 변수에 저장
train = IVF_train
test = IVF_test

타겟변수 비율 확인

In [1107]:
# '임신_성공_여부' 변수의 값 개수 계산
value_counts = train['임신_성공_여부'].value_counts()

# 각 값이 전체에서 차지하는 비율 계산
value_percentages = value_counts / len(train) * 100

# 결과 출력
print("임신_성공_여부 값 개수:")
print(value_counts)
print("\n임신_성공_여부 값 비율 (%):")
print(value_percentages)

임신_성공_여부 값 개수:
임신_성공_여부
0    184643
1     65417
Name: count, dtype: int64

임신_성공_여부 값 비율 (%):
임신_성공_여부
0    73.839479
1    26.160521
Name: count, dtype: float64


EDA를 통해 무조건 불임인 데이터를 제거하여 EDA 재진행

In [1108]:
train = train[train['시술_당시_나이'] != '알 수 없음']
train = train[~train['특정_시술_유형'].isin(['FER', 'GIFT'])]
train = train[train['불임_원인_-_자궁경부_문제'] != 1]
train = train[train['불임_원인_-_정자_면역학적_요인'] != 1]

values_to_drop = ['기증용', '기증용, 난자 저장용', '기증용, 배아 저장용', '난자 저장용', '난자 저장용, 배아 저장용']
train = train[~train['배아_생성_주요_이유'].isin(values_to_drop)]

train = train[train['저장된_배아_수'] < 25]
train = train[train['해동_난자_수'] < 24]
train = train[train['저장된_신선_난자_수'] < 20]
train = train[train['정자_출처'] != '배우자 및 기증 제공']
train = train[train['난자_해동_경과일'] != 1]
train = train[train['난자_혼합_경과일'] < 6]
train = train[train['배아_해동_경과일'] != 7]
train = train[~((train['착상_전_유전_진단_사용_여부'] == 1) & (train['착상_전_유전_검사_사용_여부'] == 1))]
train = train[~((train['남성_주_불임_원인'] == 1) & (train['남성_부_불임_원인'] == 1))]
train = train[~((train['여성_주_불임_원인'] == 1) & (train['여성_부_불임_원인'] == 1))]
train = train[~((train['부부_주_불임_원인'] == 1) & (train['부부_부_불임_원인'] == 1))]
train = train[train['총_임신_횟수'] != '6회 이상']
train = train[train['총_출산_횟수'] != '4회']
train = train[train['DI_임신_횟수'] != '4회']
train = train[train['DI_출산_횟수'] != '3회']
train = train[train['총_생성_배아_수'] < 38]
train = train[train['미세주입된_난자_수'] < 44]
train = train[train['해동된_배아_수'] < 11]
train = train[train['수집된_신선_난자_수'] < 51]
train = train[~((train['신선_배아_사용_여부'] == 0) & (train['동결_배아_사용_여부'] == 0) & (train['기증_배아_사용_여부'] == 0))]

test 데이터는 해당 조건일때 불임으로 예측값 뽑도록 할예정(제거는 안함)

In [1109]:
# test = test[test['시술_당시_나이'] != '알 수 없음']
# test = test[~test['특정_시술_유형'].isin(['FER', 'GIFT'])]
# test = test[test['불임_원인_-_자궁경부_문제'] != 1]
# test = test[test['불임_원인_-_정자_면역학적_요인'] != 1]

# values_to_drop = ['기증용', '기증용, 난자 저장용', '기증용, 배아 저장용', '난자 저장용', '난자 저장용, 배아 저장용']
# test = test[~test['배아_생성_주요_이유'].isin(values_to_drop)]

# test = test[test['저장된_배아_수'] < 25]
# test = test[test['해동_난자_수'] < 24]
# test = test[test['저장된_신선_난자_수'] < 20]
# test = test[test['정자_출처'] != '배우자 및 기증 제공']
# test = test[test['난자_해동_경과일'] != 1]
# test = test[test['난자_혼합_경과일'] < 6]
# test = test[test['배아_해동_경과일'] != 7]
# test = test[~((test['착상_전_유전_진단_사용_여부'] == 1) & (test['착상_전_유전_검사_사용_여부'] == 1))]
# test = test[~((test['남성_주_불임_원인'] == 1) & (test['남성_부_불임_원인'] == 1))]
# test = test[~((test['여성_주_불임_원인'] == 1) & (test['여성_부_불임_원인'] == 1))]
# test = test[~((test['부부_주_불임_원인'] == 1) & (test['부부_부_불임_원인'] == 1))]
# test = test[test['총_임신_횟수'] != '6회 이상']
# test = test[test['총_출산_횟수'] != '4회']
# test = test[test['DI_임신_횟수'] != '4회']
# test = test[test['DI_출산_횟수'] != '3회']
# test = test[test['총_생성_배아_수'] < 38]
# test = test[test['미세주입된_난자_수'] < 44]
# test = test[test['해동된_배아_수'] < 11]
# test = test[test['수집된_신선_난자_수'] < 51]
# test = test[~((test['신선_배아_사용_여부'] == 0) & (test['동결_배아_사용_여부'] == 0) & (test['기증_배아_사용_여부'] == 0))]

### 1. 시술_시기_코드

난임 시술을 받은 시기를 기준으로 코드 부여

In [1110]:
value_counts_ratio(train, '시술_시기_코드')


시술_시기_코드별 임신_성공_여부 비율 및 갯수

                 0         1      0     1  Total
variable                                        
TRCMWS    0.728477  0.271523  21729  8099  29828
TRDQAZ    0.745594  0.254406  22423  7651  30074
TRJXFG    0.713041  0.286959  20460  8234  28694
TRVNRY    0.717632  0.282368  20741  8161  28902
TRXQMD    0.717643  0.282357  19634  7725  27359
TRYBLT    0.713980  0.286020  20709  8296  29005
TRZKPL    0.720453  0.279547  20378  7907  28285


In [1111]:
# '시술_시기_코드' 변수 제거
train = train.drop(columns=['시술_시기_코드'])
test = test.drop(columns=['시술_시기_코드'])

### 2. 시술 당시 나이

환자의 시술 당시 나이(연령대)

In [1112]:
len(train)

202147

In [1113]:
value_counts_ratio(train, '시술_당시_나이')


시술_당시_나이별 임신_성공_여부 비율 및 갯수

                 0         1      0      1  Total
variable                                         
만18-34세   0.656443  0.343557  55516  29055  84571
만35-37세   0.701192  0.298808  31696  13507  45203
만38-39세   0.769702  0.230298  23265   6961  30226
만40-42세   0.836651  0.163349  23786   4644  28430
만43-44세   0.883762  0.116238   8006   1053   9059
만45-50세   0.816874  0.183126   3805    853   4658


In [1114]:
# '시술_당시_나이' 변수의 값을 중앙값으로 변환하는 함수 정의
def convert_age_category(age_category):
    if age_category == '만18-34세':
        return (18 + 34) / 2
    elif age_category == '만35-37세':
        return (35 + 37) / 2
    elif age_category == '만38-39세':
        return (38 + 39) / 2
    elif age_category == '만40-42세':
        return (40 + 42) / 2
    elif age_category == '만43-44세':
        return (43 + 44) / 2
    elif age_category == '만45-50세':
        return (45 + 50) / 2
    else:
        return None

# '시술_당시_나이' 변수의 값을 중앙값으로 변환
train['시술_당시_나이'] = train['시술_당시_나이'].apply(convert_age_category)
test['시술_당시_나이'] = test['시술_당시_나이'].apply(convert_age_category)

In [1115]:
value_counts_ratio(train, '시술_당시_나이')


시술_당시_나이별 임신_성공_여부 비율 및 갯수

                 0         1      0      1  Total
variable                                         
26.0      0.656443  0.343557  55516  29055  84571
36.0      0.701192  0.298808  31696  13507  45203
38.5      0.769702  0.230298  23265   6961  30226
41.0      0.836651  0.163349  23786   4644  28430
43.5      0.883762  0.116238   8006   1053   9059
47.5      0.816874  0.183126   3805    853   4658


### 3. 임신 시도 또는 마지막 임신 경과 일수

환자가 처음 임신을 시도한 시점 또는 마지막 임신 이후 현재까지의 경과 연수 (년 단위)

In [1116]:
len(train)

202147

In [1117]:
value_counts_ratio(train, '임신_시도_또는_마지막_임신_경과_연수')


임신_시도_또는_마지막_임신_경과_연수별 임신_성공_여부 비율 및 갯수

                 0         1    0    1  Total
variable                                     
0.0       0.500000  0.500000    4    4      8
1.0       0.666667  0.333333    2    1      3
2.0       0.823529  0.176471   14    3     17
3.0       0.767442  0.232558   33   10     43
4.0       0.744186  0.255814  128   44    172
5.0       0.706186  0.293814  274  114    388
6.0       0.685771  0.314229  347  159    506
7.0       0.708609  0.291391  428  176    604
8.0       0.747458  0.252542  441  149    590
9.0       0.753507  0.246493  376  123    499
10.0      0.750000  0.250000  327  109    436
11.0      0.760252  0.239748  241   76    317
12.0      0.716599  0.283401  177   70    247
13.0      0.789030  0.210970  187   50    237
14.0      0.753247  0.246753  116   38    154
15.0      0.740157  0.259843   94   33    127
16.0      0.849462  0.150538   79   14     93
17.0      0.753846  0.246154   49   16     65
18.0      0.821429  0.178571   46   10

In [1118]:
print("결측값 개수:", train['임신_시도_또는_마지막_임신_경과_연수'].isnull().sum())

train['임신_시도_또는_마지막_임신_경과_연수'].fillna(-1, inplace=True)
test['임신_시도_또는_마지막_임신_경과_연수'].fillna(-1, inplace=True)

print("결측값 개수:", train['임신_시도_또는_마지막_임신_경과_연수'].isnull().sum())

결측값 개수: 197500
결측값 개수: 0


In [1119]:
value_counts_ratio(train, '임신_시도_또는_마지막_임신_경과_연수')


임신_시도_또는_마지막_임신_경과_연수별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-1.0      0.722213  0.277787  142637  54863  197500
 0.0      0.500000  0.500000       4      4       8
 1.0      0.666667  0.333333       2      1       3
 2.0      0.823529  0.176471      14      3      17
 3.0      0.767442  0.232558      33     10      43
 4.0      0.744186  0.255814     128     44     172
 5.0      0.706186  0.293814     274    114     388
 6.0      0.685771  0.314229     347    159     506
 7.0      0.708609  0.291391     428    176     604
 8.0      0.747458  0.252542     441    149     590
 9.0      0.753507  0.246493     376    123     499
 10.0     0.750000  0.250000     327    109     436
 11.0     0.760252  0.239748     241     76     317
 12.0     0.716599  0.283401     177     70     247
 13.0     0.789030  0.210970     187     50     237
 14.0     0.753247  0.246753     116     38     154
 15.0     0.740157  0.

### 5. 특정_시술_유형

IVF – 체외 수정 (시험관 아기 시술)  
ICSI – 세포질 내 정자 주입  
IUI – 인공 수정 (자궁 내 주입)  
BLASTOCYST – 배반포 단계에 이식  
AH – 배아를 hatching 시켜주는 과정  
Unknown – 시술 유형 정보가 제공되지 않았거나 알 수 없음  

(*) 특수 기호의 의미  
"/" – 복합적인 시술 방법을 의미 (예: ICSI / BLASTOCYST)  
":" – 동일 시술 내 세부 조합을 의미 (예: ICSI:IVF)  

In [1120]:
value_counts_ratio(train, '특정_시술_유형')


특정_시술_유형별 임신_성공_여부 비율 및 갯수

                                            0         1        0        1  \
variable                                                                    
ICSI                                 0.723260  0.276740  83308.0  31876.0   
ICSI / AH                            0.776521  0.223479    549.0    158.0   
ICSI / AH:Unknown                    0.500000  0.500000      1.0      1.0   
ICSI / BLASTOCYST                    0.610380  0.389620    788.0    503.0   
ICSI / BLASTOCYST :IVF / BLASTOCYST  0.750000  0.250000      3.0      1.0   
ICSI / BLASTOCYST:IVF / BLASTOCYST   0.666667  0.333333      4.0      2.0   
ICSI:ICSI                            0.991007  0.008993   2204.0     20.0   
ICSI:IVF                             0.763189  0.236811    651.0    202.0   
ICSI:Unknown                         0.895522  0.104478    180.0     21.0   
IVF                                  0.711932  0.288068  56081.0  22692.0   
IVF / AH                             0.812287  

In [1121]:
test['특정_시술_유형'].value_counts()

특정_시술_유형
ICSI                                  43237
IVF                                   32350
Unknown                                9216
ICSI:ICSI                               797
ICSI / BLASTOCYST                       584
IVF:IVF                                 406
IVF / BLASTOCYST                        389
ICSI:IVF                                298
ICSI / AH                               264
IVF:ICSI                                133
IVF / AH                                105
ICSI:Unknown                             68
IVF:Unknown                              41
ICSI / BLASTOCYST:IVF / BLASTOCYST        2
IVF:Unknown:Unknown:Unknown               1
Name: count, dtype: int64

In [1122]:
# 시술 유형 변환 함수
def transform_treatment(treatment):
    if isinstance(treatment, str):  # treatment가 문자열인지 확인
        if treatment.startswith("ICSI"):
            if "AH" in treatment:
                return "ICSI / AH"
            elif "BLASTOCYST" in treatment:
                return "ICSI / BLASTOCYST"
            return "ICSI"
        elif treatment.startswith("IVF"):
            if "AH" in treatment:
                return "IVF / AH"
            elif "BLASTOCYST" in treatment:
                return "IVF / BLASTOCYST"
            return "IVF"
    return treatment  # 기타 시술은 그대로 반환

# 변환된 열 추가
train['특정_시술_유형'] = train['특정_시술_유형'].apply(transform_treatment)
test['특정_시술_유형'] = test['특정_시술_유형'].apply(transform_treatment)

In [1123]:
value_counts_ratio(train, '특정_시술_유형')


특정_시술_유형별 임신_성공_여부 비율 및 갯수

                          0         1      0      1   Total
variable                                                   
ICSI               0.728867  0.271133  86343  32119  118462
ICSI / AH          0.775740  0.224260    550    159     709
ICSI / BLASTOCYST  0.611068  0.388932    795    506    1301
IVF                0.715893  0.284107  57527  22830   80357
IVF / AH           0.813559  0.186441    240     55     295
IVF / BLASTOCYST   0.581140  0.418860    530    382     912
Unknown            0.798165  0.201835     87     22     109


In [1124]:
test['특정_시술_유형'].value_counts()

특정_시술_유형
ICSI                 44400
IVF                  32931
Unknown               9216
ICSI / BLASTOCYST      586
IVF / BLASTOCYST       389
ICSI / AH              264
IVF / AH               105
Name: count, dtype: int64

In [1125]:
# 더미 변수 생성 함수 정의
def create_dummy(x, keyword):
    if isinstance(x, str) and keyword in x:
        return 1
    else:
        return 0

# 더미 변수 생성
train['특정_시술_유형_ICSI_더미'] = train['특정_시술_유형'].apply(lambda x: create_dummy(x, 'ICSI'))
train['특정_시술_유형_AH_더미'] = train['특정_시술_유형'].apply(lambda x: create_dummy(x, 'AH'))
train['특정_시술_유형_BLASTOCYST_더미'] = train['특정_시술_유형'].apply(lambda x: create_dummy(x, 'BLASTOCYST'))

test['특정_시술_유형_ICSI_더미'] = test['특정_시술_유형'].apply(lambda x: create_dummy(x, 'ICSI'))
test['특정_시술_유형_AH_더미'] = test['특정_시술_유형'].apply(lambda x: create_dummy(x, 'AH'))
test['특정_시술_유형_BLASTOCYST_더미'] = test['특정_시술_유형'].apply(lambda x: create_dummy(x, 'BLASTOCYST'))

In [1126]:
summarize_group(train, ['특정_시술_유형',
                        '특정_시술_유형_AH_더미',
                        '특정_시술_유형_BLASTOCYST_더미'])

Grouped by: 특정_시술_유형, 특정_시술_유형_AH_더미, 특정_시술_유형_BLASTOCYST_더미

                       group  '임신_성공_여부' count     ratio   Total
0               (ICSI, 0, 0)             32119  0.271133  118462
1          (ICSI / AH, 1, 0)               159  0.224260     709
2  (ICSI / BLASTOCYST, 0, 1)               506  0.388932    1301
3                (IVF, 0, 0)             22830  0.284107   80357
4           (IVF / AH, 1, 0)                55  0.186441     295
5   (IVF / BLASTOCYST, 0, 1)               382  0.418860     912
6            (Unknown, 0, 0)                22  0.201835     109


### 6. 배란_자극_여부

배란 자극 치료가 사용되었는지 여부

In [1127]:
value_counts_ratio(train, '배란_자극_여부')


배란_자극_여부별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.731025  0.268975   11105   4086   15191
1         0.721929  0.278071  134969  51987  186956


### 7. 배란_유도_유형

사용된 배란 유도 방법 유형

In [1128]:
value_counts_ratio(train, '배란_유도_유형')


배란_유도_유형별 임신_성공_여부 비율 및 갯수

                   0         1         0        1   Total
variable                                                 
기록되지 않은 시행  0.721933  0.278067  134969.0  51986.0  186955
생식선 자극 호르몬  0.000000  1.000000       0.0      1.0       1
알 수 없음      0.731025  0.268975   11105.0   4086.0   15191


In [1129]:
test['배란_유도_유형'].value_counts()

배란_유도_유형
기록되지 않은 시행    68599
알 수 없음        19292
Name: count, dtype: int64

In [1130]:
# 갯수 적은 배란 유도 유형 제거
train = train.loc[~train['배란_유도_유형'].isin(['생식선 자극 호르몬', '세트로타이드 (억제제)'])]
test = test.loc[~test['배란_유도_유형'].isin(['생식선 자극 호르몬', '세트로타이드 (억제제)'])]

In [1131]:
value_counts_ratio(train, '배란_유도_유형')


배란_유도_유형별 임신_성공_여부 비율 및 갯수

                   0         1       0      1   Total
variable                                             
기록되지 않은 시행  0.721933  0.278067  134969  51986  186955
알 수 없음      0.731025  0.268975   11105   4086   15191


In [1132]:
train= train.drop(columns=['배란_유도_유형'])
test = test.drop(columns=['배란_유도_유형'])

### 8. 단일_배아_이식_여부

선택적으로 단일 배아를 이식했는지 여부

In [1133]:
value_counts_ratio(train, '단일_배아_이식_여부')


단일_배아_이식_여부별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0.0       0.757626  0.242374  115194  36852  152046
1.0       0.616367  0.383633   30880  19220   50100


### 9. 착상_전_유전_검사_사용_여부

PGS(착상 전 유전 검사)가 시행되었는지 여부

In [1134]:
value_counts_ratio(train, '착상_전_유전_검사_사용_여부')


착상_전_유전_검사_사용_여부별 임신_성공_여부 비율 및 갯수

                0        1     0    1  Total
variable                                    
1.0       0.88346  0.11654  1971  260   2231


In [1135]:
# 결측값 개수 확인
print("결측값 개수:", train['착상_전_유전_검사_사용_여부'].isnull().sum())

# 결측값을 0으로 채우기
train['착상_전_유전_검사_사용_여부'].fillna(0, inplace=True)
test['착상_전_유전_검사_사용_여부'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['착상_전_유전_검사_사용_여부'].isnull().sum())

결측값 개수: 199915
결측값 개수: 0


### 10. 착상_전_유전_진단_사용_여부

PGD(착상 전 유전 진단)가 시행되었는지 여부

In [1136]:
value_counts_ratio(train, '착상_전_유전_진단_사용_여부')


착상_전_유전_진단_사용_여부별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0.0       0.721288  0.278712  144003  55644  199647
1.0       0.828731  0.171269    2071    428    2499


In [1137]:
summarize_group(train, [
    '착상_전_유전_진단_사용_여부',
     '착상_전_유전_검사_사용_여부'
     ])

Grouped by: 착상_전_유전_진단_사용_여부, 착상_전_유전_검사_사용_여부

        group  '임신_성공_여부' count     ratio   Total
0  (0.0, 0.0)             55384  0.280545  197416
1  (0.0, 1.0)               260  0.116540    2231
2  (1.0, 0.0)               428  0.171269    2499


### 11. 남성_주_불임_원인

남성의 주요 불임 원인 여부

In [1138]:
value_counts_ratio(train, '남성_주_불임_원인')


남성_주_불임_원인별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.722324  0.277676  143387  55121  198508
1         0.738593  0.261407    2687    951    3638


### 12. 남성_부_불임_원인

남성의 부차적 불임 원인 여부

In [1139]:
value_counts_ratio(train, '남성_부_불임_원인')


남성_부_불임_원인별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.722286  0.277714  144822  55683  200505
1         0.762949  0.237051    1252    389    1641


In [1140]:
summarize_group(train, ['남성_주_불임_원인', '남성_부_불임_원인'])

Grouped by: 남성_주_불임_원인, 남성_부_불임_원인

    group  '임신_성공_여부' count     ratio   Total
0  (0, 0)             54732  0.278015  196867
1  (0, 1)               389  0.237051    1641
2  (1, 0)               951  0.261407    3638


### 13. 여성_주_불임_원인

여성의 주요 불임 원인 여부

In [1141]:
value_counts_ratio(train, '여성_주_불임_원인')


여성_주_불임_원인별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.722274  0.277726  143211  55067  198278
1         0.740176  0.259824    2863   1005    3868


### 14. 여성_부_불임_원인

여성의 부차적 불임 원인 여부

In [1142]:
value_counts_ratio(train, '여성_부_불임_원인')


여성_부_불임_원인별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.722280  0.277720  144880  55707  200587
1         0.765876  0.234124    1194    365    1559


In [1143]:
summarize_group(train, ['여성_주_불임_원인', '여성_부_불임_원인'])

Grouped by: 여성_주_불임_원인, 여성_부_불임_원인

    group  '임신_성공_여부' count     ratio   Total
0  (0, 0)             54702  0.278072  196719
1  (0, 1)               365  0.234124    1559
2  (1, 0)              1005  0.259824    3868


### 15. 부부_주_불임_원인

부부의 주요 불임 원인 여부

In [1144]:
value_counts_ratio(train, '부부_주_불임_원인')


부부_주_불임_원인별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.722286  0.277714  142949  54963  197912
1         0.738073  0.261927    3125   1109    4234


### 16. 부부_부_불임_원인

부부의 부차적 불임 원인 여부

In [1145]:
value_counts_ratio(train, '부부_부_불임_원인')


부부_부_불임_원인별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.722331  0.277669  145247  55834  201081
1         0.776526  0.223474     827    238    1065


In [1146]:
summarize_group(train, ['부부_주_불임_원인', '부부_부_불임_원인'])

Grouped by: 부부_주_불임_원인, 부부_부_불임_원인

    group  '임신_성공_여부' count     ratio   Total
0  (0, 0)             54725  0.278008  196847
1  (0, 1)               238  0.223474    1065
2  (1, 0)              1109  0.261927    4234


In [1147]:
# 두 변수에 대해 그룹별 갯수 카운팅
test.groupby(['부부_주_불임_원인', '부부_부_불임_원인']).size().reset_index(name='count')

,부부_주_불임_원인,부부_부_불임_원인,count
0,0,0,84254
1,0,1,751
2,1,0,2883
3,1,1,3


### 17. 불명확_불임_원인

불명의 원인으로 인한 불임 여부

In [1148]:
value_counts_ratio(train, '불명확_불임_원인')


불명확_불임_원인별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.721079  0.278921  107200  41466  148666
1         0.726889  0.273111   38874  14606   53480


### 18. 불임_원인_-_난관_질환

난관 이상으로 인한 불임 여부

In [1149]:
value_counts_ratio(train, '불임_원인_-_난관_질환')


불임_원인_-_난관_질환별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.722378  0.277622  125896  48384  174280
1         0.724108  0.275892   20178   7688   27866


### 19. 불임_원인_-_남성_요인

남성 요인으로 인한 불임 여부

In [1150]:
value_counts_ratio(train, '불임_원인_-_남성_요인')


불임_원인_-_남성_요인별 임신_성공_여부 비율 및 갯수

                 0         1      0      1   Total
variable                                          
0         0.736012  0.263988  92329  33116  125445
1         0.700708  0.299292  53745  22956   76701


### 20. 불임_원인_-_배란_장애

배란 장애로 인한 불임 여부

In [1151]:
value_counts_ratio(train, '불임_원인_-_배란_장애')


불임_원인_-_배란_장애별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.727427  0.272573  128388  48108  176496
1         0.689513  0.310487   17686   7964   25650


### 22. 불임_원인_-_자궁경부_문제

자궁경부 이상으로 인한 불임 여부

In [1152]:
value_counts_ratio(train, '불임_원인_-_자궁경부_문제')


불임_원인_-_자궁경부_문제별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.722616  0.277384  146074  56072  202146


In [1153]:
train = train.drop(columns=['불임_원인_-_자궁경부_문제'])
test = test.drop(columns=['불임_원인_-_자궁경부_문제'])

### 23. 불임_원인_-_자궁내막증

자궁내막증으로 인한 불임 여부

In [1154]:
value_counts_ratio(train, '불임_원인_-_자궁내막증')


불임_원인_-_자궁내막증별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.722213  0.277787  135506  52120  187626
1         0.727824  0.272176   10568   3952   14520


### 24. 불임_원인_-_정자_농도

파트너의 정자 농도 문제 여부

In [1155]:
value_counts_ratio(train, '불임_원인_-_정자_농도')


불임_원인_-_정자_농도별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.722617  0.277383  145973  56033  202006
1         0.721429  0.278571     101     39     140


### 25. 불임_원인_-_정자_면역학적_요인

파트너의 정자 면역 관련 문제 여부

In [1156]:
value_counts_ratio(train, '불임_원인_-_정자_면역학적_요인')


불임_원인_-_정자_면역학적_요인별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.722616  0.277384  146074  56072  202146


In [1157]:
train = train.drop(columns=['불임_원인_-_정자_면역학적_요인'])
test = test.drop(columns=['불임_원인_-_정자_면역학적_요인'])

### 26. 불임_원인_-_정자_운동성

파트너의 정자 운동성 문제 여부

In [1158]:
value_counts_ratio(train, '불임_원인_-_정자_운동성')


불임_원인_-_정자_운동성별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.722600  0.277400  146028  56059  202087
1         0.779661  0.220339      46     13      59


### 27. 불임_원인_-_정자_형태

파트너의 정자 형태 문제 여부

In [1159]:
value_counts_ratio(train, '불임_원인_-_정자_형태')


불임_원인_-_정자_형태별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.722606  0.277394  146012  56051  202063
1         0.746988  0.253012      62     21      83


### 28. 배아_생성_주요_이유

배아 생성의 주된 목적(시술, 보관 등)

In [1160]:
value_counts_ratio(train, '배아_생성_주요_이유')


배아_생성_주요_이유별 임신_성공_여부 비율 및 갯수

                            0         1         0        1   Total
variable                                                          
기증용, 배아 저장용, 현재 시술용  0.650000  0.350000      13.0      7.0      20
기증용, 현재 시술용          0.612134  0.387866    2260.0   1432.0    3692
난자 저장용, 배아 저장용, 연구용  1.000000  0.000000       1.0      0.0       1
난자 저장용, 현재 시술용       0.800000  0.200000       4.0      1.0       5
배아 저장용               0.999093  0.000907    8817.0      8.0    8825
배아 저장용, 현재 시술용       0.750000  0.250000      60.0     20.0      80
현재 시술용               0.711887  0.288113  134919.0  54604.0  189523


In [1161]:
# '연구용'이라는 값이 포함된 행 삭제
train = train[~train['배아_생성_주요_이유'].str.contains('연구용', na=False)]
test = test[~test['배아_생성_주요_이유'].str.contains('연구용', na=False)]

In [1162]:
value_counts_ratio(train, '배아_생성_주요_이유')


배아_생성_주요_이유별 임신_성공_여부 비율 및 갯수

                            0         1       0      1   Total
variable                                                      
기증용, 배아 저장용, 현재 시술용  0.650000  0.350000      13      7      20
기증용, 현재 시술용          0.612134  0.387866    2260   1432    3692
난자 저장용, 현재 시술용       0.800000  0.200000       4      1       5
배아 저장용               0.999093  0.000907    8817      8    8825
배아 저장용, 현재 시술용       0.750000  0.250000      60     20      80
현재 시술용               0.711887  0.288113  134919  54604  189523


In [1163]:
# '현재 시술용' 키워드가 포함된 경우 1, 그렇지 않은 경우 0을 부여하는 더미 변수 생성
train['배아_생성_주요_이유_현재시술용'] = train['배아_생성_주요_이유'].str.contains('현재 시술용').astype(int)
test['배아_생성_주요_이유_현재시술용'] = test['배아_생성_주요_이유'].str.contains('현재 시술용').astype(int)

# '기증용' 키워드가 포함된 경우 1, 그렇지 않은 경우 0을 부여하는 더미 변수 생성
train['배아_생성_주요_이유_기증용'] = train['배아_생성_주요_이유'].str.contains('기증용').astype(int)
test['배아_생성_주요_이유_기증용'] = test['배아_생성_주요_이유'].str.contains('기증용').astype(int)

# '난자 저장용' 키워드가 포함된 경우 1, 그렇지 않은 경우 0을 부여하는 더미 변수 생성
train['배아_생성_주요_이유_난자저장용'] = train['배아_생성_주요_이유'].str.contains('난자 저장용').astype(int)
test['배아_생성_주요_이유_난자저장용'] = test['배아_생성_주요_이유'].str.contains('난자 저장용').astype(int)

# '배아 저장용' 키워드가 포함된 경우 1, 그렇지 않은 경우 0을 부여하는 더미 변수 생성
train['배아_생성_주요_이유_배아저장용'] = train['배아_생성_주요_이유'].str.contains('배아 저장용').astype(int)
test['배아_생성_주요_이유_배아저장용'] = test['배아_생성_주요_이유'].str.contains('배아 저장용').astype(int)

In [1164]:
summarize_group(train, [
    # '배아_생성_주요_이유',
    '배아_생성_주요_이유_현재시술용',
    '배아_생성_주요_이유_기증용',
    '배아_생성_주요_이유_난자저장용',
    '배아_생성_주요_이유_배아저장용'
                        ])

Grouped by: 배아_생성_주요_이유_현재시술용, 배아_생성_주요_이유_기증용, 배아_생성_주요_이유_난자저장용, 배아_생성_주요_이유_배아저장용

          group  '임신_성공_여부' count     ratio   Total
0  (0, 0, 0, 1)                 8  0.000907    8825
1  (1, 0, 0, 0)             54604  0.288113  189523
2  (1, 0, 0, 1)                20  0.250000      80
3  (1, 0, 1, 0)                 1  0.200000       5
4  (1, 1, 0, 0)              1432  0.387866    3692
5  (1, 1, 0, 1)                 7  0.350000      20


In [1165]:
train['배아_생성_주요_이유'].isnull().sum()

0

In [1166]:
value_counts_ratio(train, '배아_생성_주요_이유_난자저장용')


배아_생성_주요_이유_난자저장용별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.722613  0.277387  146069  56071  202140
1         0.800000  0.200000       4      1       5


갯수 적으므로 제거

In [1167]:
# 변수 제거
train = train.drop(columns=['배아_생성_주요_이유_난자저장용'])
test = test.drop(columns=['배아_생성_주요_이유_난자저장용'])

In [1168]:
train = train.drop(columns=['배아_생성_주요_이유'])
test = test.drop(columns=['배아_생성_주요_이유'])

### 29. 총_시술_횟수

IVF 및 DI 포함한 전체 시술 횟수

In [1169]:
value_counts_ratio(train, '총_시술_횟수')


총_시술_횟수별 임신_성공_여부 비율 및 갯수

                 0         1      0      1  Total
variable                                         
0회        0.700138  0.299862  65626  28107  93733
1회        0.735404  0.264596  32107  11552  43659
2회        0.734880  0.265120  20426   7369  27795
3회        0.744036  0.255964  11726   4034  15760
4회        0.748109  0.251891   6626   2231   8857
5회        0.766673  0.233327   3782   1151   4933
6회 이상     0.780238  0.219762   5780   1628   7408


### 30. 클리닉_내_총_시술_횟수

특정 클리닉에서 받은 IVF 및 DI 시술 횟수

In [1170]:
value_counts_ratio(train, '클리닉_내_총_시술_횟수')


클리닉_내_총_시술_횟수별 임신_성공_여부 비율 및 갯수

                 0         1      0      1   Total
variable                                          
0회        0.706415  0.293585  80929  33634  114563
1회        0.740848  0.259152  31835  11136   42971
2회        0.740488  0.259512  16835   5900   22735
3회        0.745314  0.254686   8071   2758   10829
4회        0.745091  0.254909   3908   1337    5245
5회        0.765278  0.234722   1966    603    2569
6회 이상     0.782246  0.217754   2529    704    3233


### 31. IVF_시술_횟수

이전에 수행된 IVF 시술 횟수

In [1171]:
value_counts_ratio(train, 'IVF_시술_횟수')


IVF_시술_횟수별 임신_성공_여부 비율 및 갯수

                 0         1      0      1  Total
variable                                         
0회        0.699955  0.300045  67559  28960  96519
1회        0.735603  0.264397  32624  11726  44350
2회        0.734752  0.265248  20407   7367  27774
3회        0.748178  0.251822  11290   3800  15090
4회        0.751489  0.248511   6181   2044   8225
5회        0.775749  0.224251   3442    995   4437
6회 이상     0.794783  0.205217   4570   1180   5750


### 32. DI_시술_횟수

이전에 수행된 DI 시술 횟수

In [1172]:
value_counts_ratio(train, 'DI_시술_횟수')


DI_시술_횟수별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0회        0.722615  0.277385  141639  54370  196009
1회        0.747088  0.252912     898    304    1202
2회        0.736122  0.263878     968    347    1315
3회        0.708517  0.291483    1123    462    1585
4회        0.716690  0.283310     511    202     713
5회        0.706024  0.293976     293    122     415
6회 이상     0.707506  0.292494     641    265     906


### 33. 총_임신_횟수

IVF 및 DI 포함한 총 임신 횟수

In [1173]:
value_counts_ratio(train, '총_임신_횟수')


총_임신_횟수별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0회        0.725643  0.274357  126148  47695  173843
1회        0.702159  0.297841   17528   7435   24963
2회        0.710160  0.289840    2090    853    2943
3회        0.774011  0.225989     274     80     354
4회        0.794872  0.205128      31      8      39
5회        0.666667  0.333333       2      1       3


### 34. IVF_임신_횟수

IVF 시술로 인한 임신 횟수

In [1174]:
value_counts_ratio(train, 'IVF_임신_횟수')


IVF_임신_횟수별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0회        0.725513  0.274487  126499  47859  174358
1회        0.702084  0.297916   17246   7318   24564
2회        0.715039  0.284961    2035    811    2846
3회        0.776119  0.223881     260     75     335
4회        0.794872  0.205128      31      8      39
5회        0.666667  0.333333       2      1       3


### 35. DI_임신_횟수

DI 시술로 인한 임신 횟수

In [1175]:
value_counts_ratio(train, 'DI_임신_횟수')


DI_임신_횟수별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0회        0.722772  0.277228  145687  55880  201567
1회        0.670565  0.329435     344    169     513
2회        0.620690  0.379310      36     22      58
3회        0.857143  0.142857       6      1       7


### 36. 총_출산_횟수

IVF 및 DI 시술로 출산한 횟수

In [1176]:
value_counts_ratio(train, '총_출산_횟수')


총_출산_횟수별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0회        0.725569  0.274431  131960  49911  181871
1회        0.696871  0.303129   13451   5851   19302
2회        0.679204  0.320796     614    290     904
3회        0.705882  0.294118      48     20      68


### 37. IVF_출산_횟수

IVF 시술로 출산한 횟수

In [1177]:
value_counts_ratio(train, 'IVF_출산_횟수')


IVF_출산_횟수별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0회        0.725442  0.274558  132230  50045  182275
1회        0.696970  0.303030   13202   5740   18942
2회        0.689535  0.310465     593    267     860
3회        0.705882  0.294118      48     20      68


### 38. DI_출산_횟수

DI 시술로 출산한 횟수

In [1178]:
value_counts_ratio(train, 'DI_출산_횟수')


DI_출산_횟수별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0회        0.722762  0.277238  145797  55925  201722
1회        0.655779  0.344221     261    137     398
2회        0.600000  0.400000      15     10      25


In [1179]:
# 숫자 매핑
mapping = {
    '0회': 0, '1회': 1, '2회': 2, '3회': 3, 
    '4회': 4, '5회': 5, '6회 이상': 6
}

# 변환할 열 목록
columns_to_map = [
    '총_시술_횟수', '클리닉_내_총_시술_횟수', 'IVF_시술_횟수', 
    'DI_시술_횟수', '총_임신_횟수', 'IVF_임신_횟수', 'DI_임신_횟수', 
    '총_출산_횟수', 'IVF_출산_횟수', 'DI_출산_횟수'
]

# 숫자로 변환
for col in columns_to_map:
    train[col] = train[col].map(mapping)
    test[col] = test[col].map(mapping)

In [1180]:
# 비율 계산 (train 데이터프레임)
train['IVF_임신_시술_비율'] = np.where(train['IVF_시술_횟수'] > 0, train['IVF_임신_횟수'] / train['IVF_시술_횟수'], 0)
train['DI_임신_시술_비율'] = np.where(train['DI_시술_횟수'] > 0, train['DI_임신_횟수'] / train['DI_시술_횟수'], 0)
train['임신_성공률'] = np.where(train['총_시술_횟수'] > 0, train['총_임신_횟수'] / train['총_시술_횟수'], 0)

# 비율 계산 (test 데이터프레임)
test['IVF_임신_시술_비율'] = np.where(test['IVF_시술_횟수'] > 0, test['IVF_임신_횟수'] / test['IVF_시술_횟수'], 0)
test['DI_임신_시술_비율'] = np.where(test['DI_시술_횟수'] > 0, test['DI_임신_횟수'] / test['DI_시술_횟수'], 0)
test['임신_성공률'] = np.where(test['총_시술_횟수'] > 0, test['총_임신_횟수'] / test['총_시술_횟수'], 0)

In [1181]:
# 변수 리스트
columns = [
    # '클리닉_내_총_시술_횟수', 
    '총_시술_횟수', '총_출산_횟수', '총_임신_횟수',
    # 'IVF_시술_횟수', 'IVF_임신_횟수', 'IVF_출산_횟수', 'IVF_임신_시술_비율',
    'DI_시술_횟수', 'DI_임신_횟수', 'DI_출산_횟수',
    'DI_임신_시술_비율', '임신_성공률'
]

# 변수 제거
train = train.drop(columns=columns)
test = test.drop(columns=columns)

### 39. 총_생성_배아_수

해당 시술에서 생성된 배아의 총 개수

In [1182]:
value_counts_ratio(train, '총_생성_배아_수')


총_생성_배아_수별 임신_성공_여부 비율 및 갯수

                 0         1        0       1  Total
variable                                            
0.0       0.997144  0.002856   6633.0    19.0   6652
1.0       0.904945  0.095055  13671.0  1436.0  15107
2.0       0.819391  0.180609  15643.0  3448.0  19091
3.0       0.757997  0.242003  15805.0  5046.0  20851
4.0       0.722277  0.277723  15521.0  5968.0  21489
5.0       0.693951  0.306049  14099.0  6218.0  20317
6.0       0.676382  0.323618  12532.0  5996.0  18528
7.0       0.660662  0.339338  10525.0  5406.0  15931
8.0       0.649823  0.350177   8809.0  4747.0  13556
9.0       0.644689  0.355311   7138.0  3934.0  11072
10.0      0.638710  0.361290   5765.0  3261.0   9026
11.0      0.632832  0.367168   4464.0  2590.0   7054
12.0      0.646365  0.353635   3672.0  2009.0   5681
13.0      0.632791  0.367209   2721.0  1579.0   4300
14.0      0.643221  0.356779   2149.0  1192.0   3341
15.0      0.642143  0.357857   1606.0   895.0   2501
16.0      0.6512

### 40. 미세주입된_난자_수

정자 미세주입(ICSI)으로 처리된 난자의 수

In [1183]:
value_counts_ratio(train, '미세주입된_난자_수')


미세주입된_난자_수별 임신_성공_여부 비율 및 갯수

                 0         1      0      1  Total
variable                                         
0.0       0.717229  0.282771  57757  22771  80528
1.0       0.942444  0.057556   4241    259   4500
2.0       0.888321  0.111679   5560    699   6259
3.0       0.832761  0.167239   6543   1314   7857
4.0       0.788968  0.211032   7552   2020   9572
5.0       0.744061  0.255939   7861   2704  10565
6.0       0.722873  0.277127   7961   3052  11013
7.0       0.704036  0.295964   7291   3065  10356
8.0       0.692930  0.307070   6684   2962   9646
9.0       0.681271  0.318729   5831   2728   8559
10.0      0.664330  0.335670   5118   2586   7704
11.0      0.649769  0.350231   4217   2273   6490
12.0      0.659152  0.340848   3684   1905   5589
13.0      0.641208  0.358792   2972   1663   4635
14.0      0.660654  0.339346   2527   1298   3825
15.0      0.650394  0.349606   2065   1110   3175
16.0      0.657244  0.342756   1674    873   2547
17.0      0.658637 

### 41. 미세주입에서_생성된_배아_수

ICSI를 통해 생성된 배아의 수

In [1184]:
value_counts_ratio(train, '미세주입에서_생성된_배아_수')


미세주입에서_생성된_배아_수별 임신_성공_여부 비율 및 갯수

                 0         1        0        1  Total
variable                                             
0.0       0.730528  0.269472  61791.0  22793.0  84584
1.0       0.908278  0.091722   8942.0    903.0   9845
2.0       0.817256  0.182744  10268.0   2296.0  12564
3.0       0.758001  0.241999  10421.0   3327.0  13748
4.0       0.719261  0.280739   9933.0   3877.0  13810
5.0       0.689103  0.310897   8866.0   4000.0  12866
6.0       0.670948  0.329052   7732.0   3792.0  11524
7.0       0.658276  0.341724   6224.0   3231.0   9455
8.0       0.650665  0.349335   5042.0   2707.0   7749
9.0       0.640274  0.359726   3930.0   2208.0   6138
10.0      0.635020  0.364980   3050.0   1753.0   4803
11.0      0.623722  0.376278   2319.0   1399.0   3718
12.0      0.661492  0.338508   1960.0   1003.0   2963
13.0      0.640954  0.359046   1371.0    768.0   2139
14.0      0.644988  0.355012   1081.0    595.0   1676
15.0      0.654424  0.345576    784.0    414.0

### 42. 이식된_배아_수

이식된 배아의 총 수

In [1185]:
value_counts_ratio(train, '이식된_배아_수')


이식된_배아_수별 임신_성공_여부 비율 및 갯수

                 0         1      0      1  Total
variable                                         
0.0       0.999049  0.000951  26275     25  26300
1.0       0.665682  0.334318  49821  25021  74842
2.0       0.680494  0.319506  63175  29662  92837
3.0       0.832966  0.167034   6802   1364   8166


### 43. 미세주입_배아_이식_수

ICSI 배아 중 이식된 배아의 수

In [1186]:
value_counts_ratio(train, '미세주입_배아_이식_수')


미세주입_배아_이식_수별 임신_성공_여부 비율 및 갯수

                 0         1      0      1  Total
variable                                         
0.0       0.761412  0.238588  74412  23317  97729
1.0       0.672313  0.327687  29091  14179  43270
2.0       0.684115  0.315885  38450  17754  56204
3.0       0.833671  0.166329   4120    822   4942


### 44. 저장된_배아_수

환자를 위해 저장된 배아의 수

In [1187]:
value_counts_ratio(train, '저장된_배아_수')


저장된_배아_수별 임신_성공_여부 비율 및 갯수

                 0         1        0        1   Total
variable                                              
0.0       0.766220  0.233780  92613.0  28257.0  120870
1.0       0.653381  0.346619  12803.0   6792.0   19595
2.0       0.628922  0.371078  12247.0   7226.0   19473
3.0       0.630212  0.369788   8373.0   4913.0   13286
4.0       0.637978  0.362022   5819.0   3302.0    9121
5.0       0.649381  0.350619   3832.0   2069.0    5901
6.0       0.666257  0.333743   2713.0   1359.0    4072
7.0       0.703065  0.296935   1835.0    775.0    2610
8.0       0.711892  0.288108   1317.0    533.0    1850
9.0       0.739370  0.260630    939.0    331.0    1270
10.0      0.778664  0.221336    781.0    222.0    1003
11.0      0.827642  0.172358    509.0    106.0     615
12.0      0.839695  0.160305    440.0     84.0     524
13.0      0.912596  0.087404    355.0     34.0     389
14.0      0.937500  0.062500    315.0     21.0     336
15.0      0.928000  0.072000    232.

### 45. 미세주입_후_저장된_배아_수

ICSI 처리된 배아 중 저장된 배아의 수

In [1188]:
value_counts_ratio(train, '미세주입_후_저장된_배아_수')


미세주입_후_저장된_배아_수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
0.0       0.741132  0.258868  116165.0  40575.0  156740
1.0       0.653060  0.346940    8015.0   4258.0   12273
2.0       0.623938  0.376062    7053.0   4251.0   11304
3.0       0.632805  0.367195    4672.0   2711.0    7383
4.0       0.644127  0.355873    3115.0   1721.0    4836
5.0       0.654655  0.345345    1962.0   1035.0    2997
6.0       0.671005  0.328995    1289.0    632.0    1921
7.0       0.740095  0.259905     934.0    328.0    1262
8.0       0.739583  0.260417     639.0    225.0     864
9.0       0.766340  0.233660     469.0    143.0     612
10.0      0.829374  0.170626     384.0     79.0     463
11.0      0.848592  0.151408     241.0     43.0     284
12.0      0.854331  0.145669     217.0     37.0     254
13.0      0.928205  0.071795     181.0     14.0     195
14.0      0.970760  0.029240     166.0      5.0     171
15.0      0.

### 46. 해동된_배아_수

동결 해동된 배아의 수

In [1189]:
value_counts_ratio(train, '해동된_배아_수')


해동된_배아_수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
0.0       0.722274  0.277726  145596.0  55984.0  201580
1.0       0.851974  0.148026     259.0     45.0     304
2.0       0.814159  0.185841      92.0     21.0     113
3.0       0.780000  0.220000      39.0     11.0      50
4.0       0.900000  0.100000      27.0      3.0      30
5.0       0.863636  0.136364      19.0      3.0      22
6.0       0.937500  0.062500      15.0      1.0      16
7.0       0.818182  0.181818       9.0      2.0      11
8.0       0.875000  0.125000       7.0      1.0       8
9.0       1.000000  0.000000       4.0      0.0       4
10.0      0.857143  0.142857       6.0      1.0       7


### 47. 해동_난자_수

동결 해동된 난자의 수

In [1190]:
value_counts_ratio(train, '해동_난자_수')


해동_난자_수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
0.0       0.722263  0.277737  144987.0  55753.0  200740
1.0       0.954545  0.045455      21.0      1.0      22
2.0       0.903226  0.096774      56.0      6.0      62
3.0       0.907692  0.092308      59.0      6.0      65
4.0       0.888889  0.111111     112.0     14.0     126
5.0       0.858491  0.141509      91.0     15.0     106
6.0       0.796407  0.203593     133.0     34.0     167
7.0       0.711268  0.288732     101.0     41.0     142
8.0       0.703196  0.296804     154.0     65.0     219
9.0       0.776699  0.223301      80.0     23.0     103
10.0      0.761364  0.238636      67.0     21.0      88
11.0      0.761194  0.238806      51.0     16.0      67
12.0      0.652174  0.347826      45.0     24.0      69
13.0      0.806452  0.193548      25.0      6.0      31
14.0      0.600000  0.400000      24.0     16.0      40
15.0      0.583333  

### 48. 수집된_신선_난자_수

신선 난자가 수집된 수

In [1191]:
value_counts_ratio(train, '수집된_신선_난자_수')


수집된_신선_난자_수별 임신_성공_여부 비율 및 갯수

                 0         1       0       1  Total
variable                                           
0.0       0.664549  0.335451  7530.0  3801.0  11331
1.0       0.942268  0.057732  4064.0   249.0   4313
2.0       0.904575  0.095425  5536.0   584.0   6120
3.0       0.863104  0.136896  6784.0  1076.0   7860
4.0       0.820049  0.179951  7984.0  1752.0   9736
5.0       0.786444  0.213556  8853.0  2404.0  11257
6.0       0.769997  0.230003  9511.0  2841.0  12352
7.0       0.742572  0.257428  9522.0  3301.0  12823
8.0       0.720971  0.279029  9413.0  3643.0  13056
9.0       0.713622  0.286378  9068.0  3639.0  12707
10.0      0.692443  0.307557  8650.0  3842.0  12492
11.0      0.677524  0.322476  7967.0  3792.0  11759
12.0      0.665920  0.334080  7138.0  3581.0  10719
13.0      0.659060  0.340940  6383.0  3302.0   9685
14.0      0.662141  0.337859  5852.0  2986.0   8838
15.0      0.656847  0.343153  5017.0  2621.0   7638
16.0      0.656538  0.343462  43

### 49. 저장된_신선_난자_수

저장된 신선 난자의 수

In [1192]:
value_counts_ratio(train, '저장된_신선_난자_수')


저장된_신선_난자_수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
0.0       0.722461  0.277539  145857.0  56032.0  201889
1.0       0.692308  0.307692       9.0      4.0      13
2.0       0.923077  0.076923      12.0      1.0      13
3.0       0.884615  0.115385      23.0      3.0      26
4.0       0.846154  0.153846      22.0      4.0      26
5.0       0.833333  0.166667      30.0      6.0      36
6.0       0.833333  0.166667      20.0      4.0      24
7.0       0.840000  0.160000      21.0      4.0      25
8.0       0.941176  0.058824      16.0      1.0      17
9.0       0.750000  0.250000       9.0      3.0      12
10.0      0.937500  0.062500      15.0      1.0      16
11.0      0.714286  0.285714      10.0      4.0      14
12.0      0.666667  0.333333       4.0      2.0       6
13.0      1.000000  0.000000       9.0      0.0       9
14.0      0.900000  0.100000       9.0      1.0      10
15.0      1.0000

### 50. 혼합된_난자_수

정자와 혼합된 총 난자 수

In [1193]:
value_counts_ratio(train, '혼합된_난자_수')


혼합된_난자_수별 임신_성공_여부 비율 및 갯수

                 0         1        0       1  Total
variable                                            
0.0       0.891892  0.108108     33.0     4.0     37
1.0       0.938977  0.061023   6001.0   390.0   6391
2.0       0.891645  0.108355   7908.0   961.0   8869
3.0       0.837277  0.162723   9200.0  1788.0  10988
4.0       0.793981  0.206019  10737.0  2786.0  13523
5.0       0.749838  0.250162  11564.0  3858.0  15422
6.0       0.735677  0.264323  11968.0  4300.0  16268
7.0       0.714169  0.285831  11346.0  4541.0  15887
8.0       0.697771  0.302229  10581.0  4583.0  15164
9.0       0.689228  0.310772   9674.0  4362.0  14036
10.0      0.671537  0.328463   8687.0  4249.0  12936
11.0      0.660827  0.339173   7620.0  3911.0  11531
12.0      0.659887  0.340113   6684.0  3445.0  10129
13.0      0.645737  0.354263   5605.0  3075.0   8680
14.0      0.658378  0.341622   4951.0  2569.0   7520
15.0      0.656110  0.343890   4247.0  2226.0   6473
16.0      0.64802

### 51. 파트너_정자와_혼합된_난자_수

파트너 정자와 혼합된 난자의 수

In [1194]:
value_counts_ratio(train, '파트너_정자와_혼합된_난자_수')


파트너_정자와_혼합된_난자_수별 임신_성공_여부 비율 및 갯수

                 0         1        0       1  Total
variable                                            
0.0       0.720747  0.279253  11648.0  4513.0  16161
1.0       0.939020  0.060980   5328.0   346.0   5674
2.0       0.891703  0.108297   7147.0   868.0   8015
3.0       0.837406  0.162594   8395.0  1630.0  10025
4.0       0.796031  0.203969   9827.0  2518.0  12345
5.0       0.750636  0.249364  10623.0  3529.0  14152
6.0       0.737268  0.262732  11017.0  3926.0  14943
7.0       0.716580  0.283420  10485.0  4147.0  14632
8.0       0.698378  0.301622   9771.0  4220.0  13991
9.0       0.690674  0.309326   8976.0  4020.0  12996
10.0      0.673357  0.326643   8052.0  3906.0  11958
11.0      0.659880  0.340120   7033.0  3625.0  10658
12.0      0.660008  0.339992   6212.0  3200.0   9412
13.0      0.646200  0.353800   5178.0  2835.0   8013
14.0      0.659143  0.340857   4614.0  2386.0   7000
15.0      0.656214  0.343786   3934.0  2061.0   5995
16.0     

### 52. 기증자_정자와_혼합된_난자_수

기증자 정자와 혼합된 난자의 수

In [1195]:
value_counts_ratio(train, '기증자_정자와_혼합된_난자_수')


기증자_정자와_혼합된_난자_수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
0.0       0.722886  0.277114  135017.0  51758.0  186775
1.0       0.936552  0.063448     679.0     46.0     725
2.0       0.886836  0.113164     768.0     98.0     866
3.0       0.834694  0.165306     818.0    162.0     980
4.0       0.770519  0.229481     920.0    274.0    1194
5.0       0.736272  0.263728     952.0    341.0    1293
6.0       0.713969  0.286031     966.0    387.0    1353
7.0       0.689711  0.310289     858.0    386.0    1244
8.0       0.691443  0.308557     800.0    357.0    1157
9.0       0.671000  0.329000     671.0    329.0    1000
10.0      0.645022  0.354978     596.0    328.0     924
11.0      0.665845  0.334155     540.0    271.0     811
12.0      0.653030  0.346970     431.0    229.0     660
13.0      0.632997  0.367003     376.0    218.0     594
14.0      0.643312  0.356688     303.0    168.0     471
15.0      0

### 53. 난자_출처

난자 출처 정보

In [1196]:
value_counts_ratio(train, '난자_출처')


난자_출처별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
기증 제공     0.654786  0.345214    7073   3729   10802
본인 제공     0.726444  0.273556  139000  52343  191343


In [1197]:
test['난자_출처'].value_counts()

난자_출처
본인 제공    82397
기증 제공     5494
Name: count, dtype: int64

### 54. 정자_출처

정자 출처 정보

In [1198]:
value_counts_ratio(train, '정자_출처')


정자_출처별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
기증 제공     0.719249  0.280751   11034   4307   15341
미할당       0.944444  0.055556      68      4      72
배우자 제공    0.722806  0.277194  134971  51761  186732


In [1199]:
test['정자_출처'].value_counts()

정자_출처
배우자 제공         80569
기증 제공           7284
미할당               33
배우자 및 기증 제공        5
Name: count, dtype: int64

### 55. 난자_기증자_나이

난자 기증자의 등록 당시 나이

In [1200]:
value_counts_ratio(train, '난자_기증자_나이')


난자_기증자_나이별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
만20세 이하   0.727273  0.272727     152     57     209
만21-25세   0.640519  0.359481    1037    582    1619
만26-30세   0.619514  0.380486    2216   1361    3577
만31-35세   0.669416  0.330584    3003   1483    4486
알 수 없음    0.726461  0.273539  139665  52589  192254


### 56. 정자_기증자_나이

정자 기증자의 등록 당시 나이

In [1201]:
value_counts_ratio(train, '정자_기증자_나이')


정자_기증자_나이별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
만20세 이하   0.778151  0.221849     463    132     595
만21-25세   0.732024  0.267976    2535    928    3463
만26-30세   0.709416  0.290584    2102    861    2963
만31-35세   0.712907  0.287093    1994    803    2797
만36-40세   0.706649  0.293351    2115    878    2993
만41-45세   0.723129  0.276871    1507    577    2084
알 수 없음    0.722868  0.277132  135357  51893  187250


### 57. 동결_배아_사용_여부

동결 배아가 사용되었는지 여부

In [1202]:
value_counts_ratio(train, '동결_배아_사용_여부')


동결_배아_사용_여부별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0.0       0.722500  0.277500  145820  56007  201827
1.0       0.795597  0.204403     253     65     318


### 58. 신선_배아_사용_여부

신선 배아가 사용되었는지 여부

In [1203]:
value_counts_ratio(train, '신선_배아_사용_여부')


신선_배아_사용_여부별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0.0       0.778761  0.221239      88     25     113
1.0       0.722584  0.277416  145985  56047  202032


### 59. 기증_배아_사용_여부

기증 배아가 사용되었는지 여부

In [1204]:
value_counts_ratio(train, '기증_배아_사용_여부')


기증_배아_사용_여부별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0.0       0.723361  0.276639  145162  55515  200677
1.0       0.620572  0.379428     911    557    1468


In [1205]:
summarize_group(train, ['신선_배아_사용_여부', '동결_배아_사용_여부', '기증_배아_사용_여부'])

Grouped by: 신선_배아_사용_여부, 동결_배아_사용_여부, 기증_배아_사용_여부

             group  '임신_성공_여부' count     ratio   Total
0  (0.0, 1.0, 0.0)                25  0.221239     113
1  (1.0, 0.0, 0.0)             55450  0.276753  200359
2  (1.0, 0.0, 1.0)               557  0.379428    1468
3  (1.0, 1.0, 0.0)                40  0.195122     205


### 60. 대리모_여부

환자가 대리모로 활동했는지 여부

In [1206]:
value_counts_ratio(train, '대리모_여부')


대리모_여부별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0.0       0.722634  0.277366  145787  55957  201744
1.0       0.713217  0.286783     286    115     401


### 61. PGD_시술_여부

해당 주기가 PGD 치료로 분류되는지 여부

In [1207]:
value_counts_ratio(train, 'PGD_시술_여부')


PGD_시술_여부별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
1.0       0.721859  0.278141  1103  425   1528


In [1209]:
# 결측값 개수 확인
print("결측값 개수:", train['PGD_시술_여부'].isnull().sum())

# 결측값을 0으로 채우기
train['PGD_시술_여부'].fillna(0, inplace=True)
test['PGD_시술_여부'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['PGD_시술_여부'].isnull().sum())

결측값 개수: 200617
결측값 개수: 0


In [1210]:
value_counts_ratio(train, 'PGD_시술_여부')


PGD_시술_여부별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0.0       0.722621  0.277379  144970  55647  200617
1.0       0.721859  0.278141    1103    425    1528


### 62. PGS_시술_여부

해당 주기가 PGS 치료로 분류되는지 여부

In [1211]:
value_counts_ratio(train, 'PGS_시술_여부')


PGS_시술_여부별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
1.0       0.828552  0.171448  1242  257   1499


In [1212]:
# 결측값 개수 확인
print("결측값 개수:", train['PGS_시술_여부'].isnull().sum())

# 결측값을 0으로 채우기
train['PGS_시술_여부'].fillna(0, inplace=True)
test['PGS_시술_여부'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['PGS_시술_여부'].isnull().sum())

결측값 개수: 200646
결측값 개수: 0


In [1213]:
summarize_group(train, ['PGD_시술_여부', 'PGS_시술_여부'])

Grouped by: PGD_시술_여부, PGS_시술_여부

        group  '임신_성공_여부' count     ratio   Total
0  (0.0, 0.0)             55390  0.278177  199118
1  (0.0, 1.0)               257  0.171448    1499
2  (1.0, 0.0)               425  0.278141    1528


### 63. 난자_채취_경과일

첫 시점으로부터 난자 채취까지 경과한 일수

In [1214]:
value_counts_ratio(train, '난자_채취_경과일')


난자_채취_경과일별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0.0       0.726072  0.273928  138528  52263  190791


In [1215]:
print("결측값 개수:", train['난자_채취_경과일'].isnull().sum())

train['난자_채취_경과일'].fillna(-1, inplace=True)
test['난자_채취_경과일'].fillna(-1, inplace=True)

print("결측값 개수:", train['난자_채취_경과일'].isnull().sum())

결측값 개수: 11354
결측값 개수: 0


### 64. 난자_해동_경과일

첫 시점으로부터 난자 해동까지 경과한 일수

In [1216]:
value_counts_ratio(train, '난자_해동_경과일')


난자_해동_경과일별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
0.0       0.772954  0.227046  1086  319   1405


In [1218]:
print("결측값 개수:", train['난자_해동_경과일'].isnull().sum())

train['난자_해동_경과일'].fillna(-1, inplace=True)
test['난자_해동_경과일'].fillna(-1, inplace=True)

print("결측값 개수:", train['난자_해동_경과일'].isnull().sum())

결측값 개수: 200740
결측값 개수: 0


### 65. 난자_혼합_경과일

첫 시점으로부터 난자와 정자 혼합까지 경과한 일수

In [1219]:
value_counts_ratio(train, '난자_혼합_경과일')


난자_혼합_경과일별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0.0       0.722301  0.277699  145517  55946  201463
1.0       0.845041  0.154959     409     75     484
2.0       0.752475  0.247525      76     25     101
3.0       0.720588  0.279412      49     19      68
4.0       0.600000  0.400000       3      2       5
5.0       0.791667  0.208333      19      5      24


In [1221]:
print("결측값 개수:", train['난자_혼합_경과일'].isnull().sum())

결측값 개수: 0


### 66. 배아_이식_경과일

첫 시점으로부터 배아 이식까지 경과한 일수

In [1222]:
value_counts_ratio(train, '배아_이식_경과일')


배아_이식_경과일별 임신_성공_여부 비율 및 갯수

                 0         1      0      1  Total
variable                                         
0.0       0.757576  0.242424     25      8     33
1.0       0.868421  0.131579    462     70    532
2.0       0.789978  0.210022  25160   6689  31849
3.0       0.742288  0.257712  42061  14603  56664
4.0       0.650170  0.349830   1721    926   2647
5.0       0.595311  0.404689  48347  32866  81213
6.0       0.699891  0.300109   1924    825   2749
7.0       0.597561  0.402439     49     33     82


In [1223]:
print("결측값 개수:", train['배아_이식_경과일'].isnull().sum())

train['배아_이식_경과일'].fillna(-1, inplace=True)
test['배아_이식_경과일'].fillna(-1, inplace=True)

print("결측값 개수:", train['배아_이식_경과일'].isnull().sum())

결측값 개수: 26376
결측값 개수: 0


In [1224]:
value_counts_ratio(train, '배아_이식_경과일')


배아_이식_경과일별 임신_성공_여부 비율 및 갯수

                 0         1      0      1  Total
variable                                         
-1.0      0.998029  0.001971  26324     52  26376
 0.0      0.757576  0.242424     25      8     33
 1.0      0.868421  0.131579    462     70    532
 2.0      0.789978  0.210022  25160   6689  31849
 3.0      0.742288  0.257712  42061  14603  56664
 4.0      0.650170  0.349830   1721    926   2647
 5.0      0.595311  0.404689  48347  32866  81213
 6.0      0.699891  0.300109   1924    825   2749
 7.0      0.597561  0.402439     49     33     82


### 67. 배아_해동_경과일

첫 시점으로부터 배아 해동까지 경과한 일수

In [1225]:
value_counts_ratio(train, '배아_해동_경과일')


배아_해동_경과일별 임신_성공_여부 비율 및 갯수

                 0         1      0     1  Total
variable                                        
0.0       0.777778  0.222222    7.0   2.0      9
1.0       0.818182  0.181818   27.0   6.0     33
2.0       0.833333  0.166667  145.0  29.0    174
3.0       0.886792  0.113208  141.0  18.0    159
4.0       1.000000  0.000000   11.0   0.0     11
5.0       0.791045  0.208955  106.0  28.0    134
6.0       0.821429  0.178571   23.0   5.0     28


In [1226]:
print("결측값 개수:", train['배아_해동_경과일'].isnull().sum())

train['배아_해동_경과일'].fillna(-1, inplace=True)
test['배아_해동_경과일'].fillna(-1, inplace=True)

print("결측값 개수:", train['배아_해동_경과일'].isnull().sum())

결측값 개수: 201597
결측값 개수: 0


In [1227]:
value_counts_ratio(train, '배아_해동_경과일')


배아_해동_경과일별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
-1.0      0.722297  0.277703  145613.0  55984.0  201597
 0.0      0.777778  0.222222       7.0      2.0       9
 1.0      0.818182  0.181818      27.0      6.0      33
 2.0      0.833333  0.166667     145.0     29.0     174
 3.0      0.886792  0.113208     141.0     18.0     159
 4.0      1.000000  0.000000      11.0      0.0      11
 5.0      0.791045  0.208955     106.0     28.0     134
 6.0      0.821429  0.178571      23.0      5.0      28


### 68. 임신_성공_여부

해당 시술로부터 임신 성공 여부

In [1228]:
train['임신_성공_여부'].value_counts()

임신_성공_여부
0    146073
1     56072
Name: count, dtype: int64

## 변수 상테 확인

In [1229]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 202145 entries, 0 to 256349
Data columns (total 63 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   ID                      202145 non-null  object 
 1   시술_당시_나이                202145 non-null  float64
 2   임신_시도_또는_마지막_임신_경과_연수   202145 non-null  float64
 3   특정_시술_유형                202143 non-null  object 
 4   배란_자극_여부                202145 non-null  int64  
 5   단일_배아_이식_여부             202145 non-null  float64
 6   착상_전_유전_검사_사용_여부        202145 non-null  float64
 7   착상_전_유전_진단_사용_여부        202145 non-null  float64
 8   남성_주_불임_원인              202145 non-null  int64  
 9   남성_부_불임_원인              202145 non-null  int64  
 10  여성_주_불임_원인              202145 non-null  int64  
 11  여성_부_불임_원인              202145 non-null  int64  
 12  부부_주_불임_원인              202145 non-null  int64  
 13  부부_부_불임_원인              202145 non-null  int64  
 14  불명확_불임_원인               2

In [1230]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 87891 entries, 0 to 90066
Data columns (total 62 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      87891 non-null  object 
 1   시술_당시_나이                87775 non-null  float64
 2   임신_시도_또는_마지막_임신_경과_연수   87891 non-null  float64
 3   특정_시술_유형                87891 non-null  object 
 4   배란_자극_여부                87891 non-null  int64  
 5   단일_배아_이식_여부             87891 non-null  float64
 6   착상_전_유전_검사_사용_여부        87891 non-null  float64
 7   착상_전_유전_진단_사용_여부        87891 non-null  float64
 8   남성_주_불임_원인              87891 non-null  int64  
 9   남성_부_불임_원인              87891 non-null  int64  
 10  여성_주_불임_원인              87891 non-null  int64  
 11  여성_부_불임_원인              87891 non-null  int64  
 12  부부_주_불임_원인              87891 non-null  int64  
 13  부부_부_불임_원인              87891 non-null  int64  
 14  불명확_불임_원인               87891 non-null  int

## 데이터 저장

In [1231]:
# '임신_성공_여부' 변수의 값 개수 계산
value_counts = train['임신_성공_여부'].value_counts()

# 각 값이 전체에서 차지하는 비율 계산
value_percentages = value_counts / len(train) * 100

# 결과 출력
print("임신_성공_여부 값 개수:")
print(value_counts)
print("\n임신_성공_여부 값 비율 (%):")
print(value_percentages)

임신_성공_여부 값 개수:
임신_성공_여부
0    146073
1     56072
Name: count, dtype: int64

임신_성공_여부 값 비율 (%):
임신_성공_여부
0    72.261495
1    27.738505
Name: count, dtype: float64


In [ ]:
# CSV 파일 저장
train.to_csv('../data/IVF_train_dataset_49.csv', index=False, encoding='utf-8-sig')
test.to_csv('../data/IVF_test_dataset_49.csv', index=False, encoding='utf-8-sig')

.